# Prepare data for Pipeline

## Setup and settings

In [1]:
# Importing packages
import os
import yaml
import logging
from Bio.Seq import Seq
from qiime2 import Artifact, Visualization
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired

from utils import *

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
params_path = os.path.join('..', 'params', 'ana-flavia-superlactacao.yaml')
experiment_name = 'jenneffer-vs-01'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'redemicro')
manifest_file = os.path.join(base_dir, 'data', 'raw', 'manifest', 'not-hist-vs-manifest.csv')
img_folder = os.path.abspath(os.path.join(base_dir, 'imgs'))
replace_files = False
trim = None

In [3]:
# Parameters
experiment_name = "ana-flavia-STD-NCxSTD-NR-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest/manifest-ana-flavia-STD-NCxSTD-NR.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-STD-NCxSTD-NR.tsv"
class_col = "sample-group"
classifier_file = "/home/lauro/nupeb/dados_brutos_rede_genoma/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


In [4]:
# Setting hardcoded parameters for development and debbuging
production = True
if not production:
    params_path = os.path.join('..', 'params', 'ana-flavia-hipotese-01')
    with open(params_path, 'r') as stream:
        params = yaml.safe_load(stream)
        experiment_name = params['experiment_name']
        base_dir = params['base_dir']
        manifest_file = params['manifest_file']
        replace_files = params['replace_files']

### Defining names and paths

In [5]:
# new_manifest = '/home/lauro/nupeb/redemicro/data/raw/manifest/karina-manifest.csv'
# with open(manifest_file, 'r') as oldm, open(new_manifest, 'w') as newm:
#     header = 'sample-id,absolute-filepath,direction\n'
#     newm.write(header)
#     for line in oldm.readlines()[1:]:
#         sid, forward, reverse = line[:-1].split(',')
#         fline = ','.join((sid, forward, 'forward')) + '\n'
#         rline = ','.join((sid, reverse, 'reverse')) + '\n'
#         newm.write(fline)
#         newm.write(rline)

In [6]:
# Define the output folder path
out_dir = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')
img_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name, 'imgs'))

# Create path if it not exist
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    print(f'New qiime-artifacts folder path created: {out_dir}')

# Create path if it not exist
if not os.path.isdir(img_folder):
    os.makedirs(img_folder)
    print(f'New img folder path created: {img_folder}')

# Define the output artifact full path
demux_file = os.path.join(out_dir, 'demux-paired.qza')
demux_view = os.path.join(out_dir, 'demux-paired.qzv')
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

New qiime-artifacts folder path created: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-STD-NCxSTD-NR-trim/qiime-artifacts
New img folder path created: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-STD-NCxSTD-NR-trim/imgs


## Step execution

This Step import all `fastq` files in a **QIIME2 Artifact** object and save it to a new `qza` file of `SampleData[PairedEndSequencesWithQuality]`

In [7]:
# If the demux file does not exist or if the old file will be replaced
if not os.path.isfile(demux_file) or replace_files:
    
    # Import data and create an artifact object
    artifact = Artifact.import_data(
        'SampleData[PairedEndSequencesWithQuality]', 
        manifest_file, 
        view_type='PairedEndFastqManifestPhred33')
    
    # Save the artifact object to a new qza file
    artifact.save(demux_file)

else:
    artifact = Artifact.load(demux_file)

In [8]:
if not os.path.isfile(demux_view) or replace_files: 
    # Generate e visualization of the Artifact
    demux_summary = demux.visualizers.summarize(artifact)

    # Save a new visualization file based on the qza file
    Visualization.save(demux_summary.visualization, filepath=demux_view)
    
    demux_view_obj = demux_summary.visualization
else:
    demux_view_obj = Visualization.load(demux_view)

## Step report


In [9]:
print(demux_view_obj)

<visualization: Visualization uuid: cc615010-9536-4402-b973-f7a739541f4d>


In [10]:
# Render Visualization
demux_view_obj

<visualization: Visualization uuid: cc615010-9536-4402-b973-f7a739541f4d>

[cutadapt](https://docs.qiime2.org/2022.2/plugins/available/cutadapt/trim-paired/)

We trim the forward primer and the reverse complement of the reverse primer from the forward reads. We trim the reverse primer and reverse complement of the forward primer from the reverse reads.

In [11]:
if trim:    
    forward_primer = [trim['forward_primer']] # ['CCTACGGGRSGCAGCAG']
    reverse_primer = [trim['reverse_primer']] # ['GGACTACHVGGGTWTCTAAT']
    forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
    reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

    res = trim_paired(
        demultiplexed_sequences=artifact,
        front_f=forward_primer,
        front_r=reverse_primer,
        adapter_f=reverse_reverse_complement,
        adapter_r=forward_reverse_complement,
        cores=threads,
        overlap=trim['overlap'],
        indels=False,
        match_read_wildcards=True,
        match_adapter_wildcards=True,
        error_rate=0.15,
        discard_untrimmed=True,
    ).trimmed_sequences

    demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
    demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')
    
    demux.visualizers.summarize(res).visualization
    res.save(demux_file_trim)
    Visualization.save(demux.visualizers.summarize(res).visualization, filepath=demux_view_trim)

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 6 --error-rate 0.15 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-ivu_boz8/210421121682_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-ivu_boz8/210421121682_1_L001_R2_001.fastq.gz --adapter ATTAGAWACCCBDGTAGTCC --front CCTACGGGRSGCAGCAG -A CTGCTGCSYCCCGTAGG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-slftvfbd/b6afce83-cc33-4219-ac0a-c83a609026ce/data/210421121682_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-slftvfbd/b6afce83-cc33-4219-ac0a-c83a609026ce/data/210421121682_1_L001_R2_001.fastq.gz



This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 6 --error-rate 0.15 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-ivu_boz8/210421121682_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-ivu_boz8/210421121682_1_L001_R2_001.fastq.gz --adapter ATTAGAWACCCBDGTAGTCC --front CCTACGGGRSGCAGCAG -A CTGCTGCSYCCCGTAGG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-slftvfbd/b6afce83-cc33-4219-ac0a-c83a609026ce/data/210421121682_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-slftvfbd/b6afce83-cc33-4219-ac0a-c83a609026ce/data/210421121682_1_L001_R2_001.fastq.gz
Processing reads on 6 cores in paired-end mode ...


Finished in 2.72 s (16 µs/read; 3.65 M reads/minute).

=== Summary ===

Total read pairs processed:            165,821
  Read 1 with adapter:                 164,234 (99.0%)
  Read 2 with adapter:                 162,538 (98.0%)

== Read fate breakdown ==
Pairs that were too short:                   2 (0.0%)
Pairs discarded as untrimmed:            4,837 (2.9%)
Pairs written (passing filters):       160,982 (97.1%)

Total basepairs processed:    84,568,710 bp
  Read 1:    50,575,405 bp
  Read 2:    33,993,305 bp
Total written (filtered):     75,046,538 bp (88.7%)
  Read 1:    45,805,681 bp
  Read 2:    29,240,857 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 38 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 18.4%
  G: 73.7%
  T: 7.9%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	2	2.5	

Finished in 2.32 s (16 µs/read; 3.87 M reads/minute).

=== Summary ===

Total read pairs processed:            149,412
  Read 1 with adapter:                 147,947 (99.0%)
  Read 2 with adapter:                 146,155 (97.8%)

== Read fate breakdown ==
Pairs that were too short:                   9 (0.0%)
Pairs discarded as untrimmed:            4,686 (3.1%)
Pairs written (passing filters):       144,717 (96.9%)

Total basepairs processed:    76,200,120 bp
  Read 1:    45,570,660 bp
  Read 2:    30,629,460 bp
Total written (filtered):     67,462,124 bp (88.5%)
  Read 1:    41,176,861 bp
  Read 2:    26,285,263 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 105 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 27.6%
  G: 71.4%
  T: 1.0%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	8	2.3

Finished in 5.02 s (15 µs/read; 4.10 M reads/minute).

=== Summary ===

Total read pairs processed:            343,261
  Read 1 with adapter:                 340,079 (99.1%)
  Read 2 with adapter:                 336,731 (98.1%)

== Read fate breakdown ==
Pairs that were too short:                   1 (0.0%)
Pairs discarded as untrimmed:            9,645 (2.8%)
Pairs written (passing filters):       333,615 (97.2%)

Total basepairs processed:   175,063,110 bp
  Read 1:   104,694,605 bp
  Read 2:    70,368,505 bp
Total written (filtered):    155,517,885 bp (88.8%)
  Read 1:    94,921,854 bp
  Read 2:    60,596,031 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 162 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 0.6%
  C: 56.2%
  G: 22.8%
  T: 20.4%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	38	5

...

Finished in 0.92 s (19 µs/read; 3.20 M reads/minute).

=== Summary ===

Total read pairs processed:             49,335
  Read 1 with adapter:                  48,895 (99.1%)
  Read 2 with adapter:                  48,247 (97.8%)

== Read fate breakdown ==
Pairs that were too short:                   5 (0.0%)
Pairs discarded as untrimmed:            1,514 (3.1%)
Pairs written (passing filters):        47,816 (96.9%)

Total basepairs processed:    25,160,850 bp
  Read 1:    15,047,175 bp
  Read 2:    10,113,675 bp
Total written (filtered):     22,289,755 bp (88.6%)
  Read 1:    13,605,875 bp
  Read 2:     8,683,880 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 31 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 3.2%
  C: 74.2%
  G: 19.4%
  T: 3.2%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	14	0.8

Finished in 0.62 s (25 µs/read; 2.36 M reads/minute).

=== Summary ===

Total read pairs processed:             24,189
  Read 1 with adapter:                  23,955 (99.0%)
  Read 2 with adapter:                  23,066 (95.4%)

== Read fate breakdown ==
Pairs that were too short:                  14 (0.1%)
Pairs discarded as untrimmed:            1,339 (5.5%)
Pairs written (passing filters):        22,836 (94.4%)

Total basepairs processed:    12,336,390 bp
  Read 1:     7,377,645 bp
  Read 2:     4,958,745 bp
Total written (filtered):     10,633,459 bp (86.2%)
  Read 1:     6,488,861 bp
  Read 2:     4,144,598 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 56 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 44.6%
  G: 51.8%
  T: 3.6%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	5	0.4	

Finished in 0.37 s (67 µs/read; 0.89 M reads/minute).

=== Summary ===

Total read pairs processed:              5,549
  Read 1 with adapter:                   5,499 (99.1%)
  Read 2 with adapter:                   5,356 (96.5%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:              240 (4.3%)
Pairs written (passing filters):         5,309 (95.7%)

Total basepairs processed:     2,829,990 bp
  Read 1:     1,692,445 bp
  Read 2:     1,137,545 bp
Total written (filtered):      2,474,084 bp (87.4%)
  Read 1:     1,510,232 bp
  Read 2:       963,852 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 4 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 100.0%
  G: 0.0%
  T: 0.0%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	1	0.1	1

Finished in 5.16 s (15 µs/read; 4.00 M reads/minute).

=== Summary ===

Total read pairs processed:            344,337
  Read 1 with adapter:                 340,922 (99.0%)
  Read 2 with adapter:                 337,158 (97.9%)

== Read fate breakdown ==
Pairs that were too short:                  12 (0.0%)
Pairs discarded as untrimmed:           10,517 (3.1%)
Pairs written (passing filters):       333,808 (96.9%)

Total basepairs processed:   175,611,870 bp
  Read 1:   105,022,785 bp
  Read 2:    70,589,085 bp
Total written (filtered):    155,616,970 bp (88.6%)
  Read 1:    94,986,971 bp
  Read 2:    60,629,999 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 43 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 32.6%
  G: 67.4%
  T: 0.0%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	7	5.3	

Finished in 6.15 s (15 µs/read; 4.06 M reads/minute).

=== Summary ===

Total read pairs processed:            416,322
  Read 1 with adapter:                 412,332 (99.0%)
  Read 2 with adapter:                 407,730 (97.9%)

== Read fate breakdown ==
Pairs that were too short:                  21 (0.0%)
Pairs discarded as untrimmed:           12,483 (3.0%)
Pairs written (passing filters):       403,818 (97.0%)

Total basepairs processed:   212,324,220 bp
  Read 1:   126,978,210 bp
  Read 2:    85,346,010 bp
Total written (filtered):    188,257,614 bp (88.7%)
  Read 1:   114,910,144 bp
  Read 2:    73,347,470 bp

=== First read: Adapter 1 ===

Sequence: ATTAGAWACCCBDGTAGTCC; Type: regular 3'; Length: 20; Trimmed: 75 times

Minimum overlap: 8
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Bases preceding removed adapters:
  A: 0.0%
  C: 40.0%
  G: 54.7%
  T: 5.3%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
8	19	6.4